In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import joblib

X_train=joblib.load('/content/drive/My Drive/Master_Thesis/X_train.pkl')
y_train=joblib.load('/content/drive/My Drive/Master_Thesis/y_train.pkl')
X_test=joblib.load('/content/drive/My Drive/Master_Thesis/X_test.pkl')
y_test=joblib.load('/content/drive/My Drive/Master_Thesis/y_test.pkl')

In [ ]:
import joblib
import pandas as pd

# Load the DataFrame
misclassified_df = joblib.load('/content/drive/My Drive/Master_Thesis/misclassified_df.pkl')


In [ ]:
misclassified_df

,X,y
0,i don't think i'm being irrational. i know tha...,2
1,"i should stop being myself, a disaster, a disa...",1
2,i have another meeting with my counselor tomor...,3
3,"i am sorry, but the text ""this is it"" is too s...",1
4,i am excited because i have something to look ...,1
...,...,...
269,i want to speak with someone who is not from m...,1
270,i have seen it all out here. i have dealt with...,3
271,i will not believe in depression but still say...,3
272,i am a 9-year-old from india going through a m...,3


In [ ]:
misclassified_df.y.value_counts()

y
3    116
2     72
1     69
0     17
Name: count, dtype: int64

In [ ]:
X_sampled = misclassified_df['X'].tolist()
y_sampled = misclassified_df['y'].tolist()

In [ ]:
# Verify the data
print(X_sampled[:5])  # Print the first 5 samples of X
print(y_sampled[:5])  # Print the first 5 samples of y

["i don't think i'm being irrational. i know that nobody will ever want to be with me romantically. i'm getting more depressed by the day, and i already notice my few friends distancing themselves. i can't blame them; i'm sure i'm not fun to be around. there are very few people with whom i feel like i can share how i'm really feeling, but doing so just pushes them away. i need my life to be different, not to be stuck in this worthless body. i need optimism, but there's none left in me. i don't think i'll ever be able to kill myself, so it's just going to be 0, 0, 0, maybe even 0 or 0 more years of this void. maybe i'll be struck by lightning or something. finger crossed.", 'i should stop being myself, a disaster, a disappointment, annoying, and so anxious.', "i have another meeting with my counselor tomorrow, and i'm probably going to receive the results of the depression, anxiety, and social anxiety test questionnaires i completed. we'll see how that goes!", 'i am sorry, but the text 

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY = " "


In [ ]:
import numpy as np
import pandas as pd
import os
import openai
import tensorflow as tf
import re
import joblib
from sklearn.utils import shuffle

# Set OpenAI API key
if 'OPENAI_API_KEY' in os.environ:
    openai.api_key = os.environ['OPENAI_API_KEY']
else:
    # If not set, fallback to the manually provided key
    openai.api_key = OPENAI_API_KEY

def remove_prefix(text):
    patterns = [r'^example \d+:', r'^\d+:', r'^\d', r'^\.']
    for pattern in patterns:
        text = re.sub(pattern, '', text.strip()).strip()
    return text

# Function to generate augmented data
def generate_augmented_data(instance, label, temperature, num_samples):
    augmented_data = []
    for _ in range(num_samples):
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=temperature,
            max_tokens=350,
            frequency_penalty=0.6,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in generating synthetic text based on specified emotional tones."
                },
                {
                    "role": "user",
                    "content": f"Please generate a reddit post without a title directly. Give me the post that is similar to a class {label}, using the same feeling words as the following example:\n\n{instance}\n\n"
                }
            ])
        synthetic_content = response.choices[0].message.content.strip().lower()

        synthetic_sentences = re.split(r'\d+\.\s+', synthetic_content)
        for sentence in synthetic_sentences:
            clean_sentence = remove_prefix(sentence.strip())
            if clean_sentence:
                augmented_data.append(clean_sentence)
    return augmented_data

# Number of samples needed for each class
samples_needed = {
    0: 3866 - 3120,  # Class 0: 3120 is the current count
    1: 3866 - 1007,  # Class 1: 1007 is the current count
    2: 3866 - 1116,  # Class 2: 1116 is the current count
    3: 3866 - 941    # Class 3: 941 is the current count
}

# Parameters
batch_size = 200  # Number of samples per batch
progress_file = '/content/drive/My Drive/Master_Thesis/progress.pkl'
X_file = '/content/drive/My Drive/Master_Thesis/X_augmented.pkl'
y_file = '/content/drive/My Drive/Master_Thesis/y_augmented.pkl'

# Load progress and data if they exist
if os.path.exists(progress_file):
    progress = joblib.load(progress_file)
    X_augmented = joblib.load(X_file)
    y_augmented = joblib.load(y_file)
else:
    progress = {label: 0 for label in samples_needed}
    X_augmented = []
    y_augmented = []

# Collect misclassified instances per class
misclassified_data_per_class = {0: [], 1: [], 2: [], 3: []}
for instance, label in zip(misclassified_df['X'], misclassified_df['y']):
    if label in misclassified_data_per_class:
        misclassified_data_per_class[label].append(instance)

# Generate augmented data
temperature = 0.4
with tf.device('/GPU:0'):  # Use GPU for preprocessing
    for label, misclassified_instances in misclassified_data_per_class.items():
        num_misclassified = len(misclassified_instances)
        instances_to_generate = samples_needed[label]
        samples_per_instance = max(1, instances_to_generate // num_misclassified)
        start_idx = progress[label]

        for idx, instance in enumerate(misclassified_instances[start_idx:], start=start_idx):
            print(f"Generating data for label {label} with original instance: {instance}")
            synthetic_samples = generate_augmented_data(instance, label, temperature, samples_per_instance)

            for sample in synthetic_samples:
                print(f"Generated synthetic sample: {sample} for label {label}")
                X_augmented.append(sample)
                y_augmented.append(label)

            progress[label] = idx + 1

            # Check if we reached a batch size
            if len(X_augmented) % batch_size == 0:
                # Save progress and data
                joblib.dump(progress, progress_file)
                joblib.dump(X_augmented, X_file)
                joblib.dump(y_augmented, y_file)
                print(f"Saved batch of {batch_size} samples. Progress: {progress}")

# Final save after the loop
joblib.dump(progress, progress_file)
joblib.dump(X_augmented, X_file)
joblib.dump(y_augmented, y_file)
print("Final save completed. All data generated.")




Generating data for label 0 with original instance: i am still struggling to grasp the purpose of linkedin. just returned from a shoot, feeling extremely tired and in need of a hug.
Generated synthetic sample: i'm feeling a bit lost trying to navigate the world of networking on linkedin. just got back from a long day at work, feeling completely drained and craving some comfort. for label 0
Generated synthetic sample: i'm feeling a bit lost today. just finished a long day at work and i'm completely drained. all i really want right now is a comforting hug to make me feel better. for label 0
Generated synthetic sample: i'm feeling a bit lost trying to navigate the world of networking on linkedin. just got back from a long day at work, feeling utterly drained and craving some comfort. for label 0
Generated synthetic sample: i just can't seem to understand the point of twitter. just got back from a long day at work, feeling utterly drained and craving some comfort. for label 0
Generated syn

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import os
import openai
from sklearn.utils import shuffle
import re
import joblib
# Function to remove unwanted prefixes
# Set OpenAI API key
if 'OPENAI_API_KEY' in os.environ:
    openai.api_key = os.environ['OPENAI_API_KEY']
else:
    # If not set, fallback to the manually provided key
    openai.api_key = OPENAI_API_KEY

def remove_prefix(text):
    patterns = [r'^example \d+:', r'^\d+:', r'^\d', r'^\.']
    for pattern in patterns:
        text = re.sub(pattern, '', text.strip()).strip()
    return text

# Function to generate augmented data
def generate_augmented_data(instance, label, temperature, num_samples):
    augmented_data = []
    for _ in range(num_samples):
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=temperature,
            max_tokens=350,
            frequency_penalty=0.6,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in generating synthetic text based on specified emotional tones."
                },
                {
                    "role": "user",
                    "content": f"Please generate reddit post without title directely give me the post that is similar to a class {label}, using the same feeling words as the following example:\n\n{instance}\n\n"
                }
            ])
        synthetic_content = response.choices[0].message.content.strip().lower()

        synthetic_sentences = re.split(r'\d+\.\s+', synthetic_content)
        for sentence in synthetic_sentences:
            clean_sentence = remove_prefix(sentence.strip())
            if clean_sentence:
                augmented_data.append(clean_sentence)
    return augmented_data

import joblib

# Number of samples needed for each class
samples_needed = {
    0: 3866 - 3120,  # Class 0: 3120 is the current count
    1: 3866 - 1007,  # Class 1: 1007 is the current count
    2: 3866 - 1116,  # Class 2: 1116 is the current count
    3: 3866 - 941    # Class 3: 941 is the current count
}

# Collect misclassified instances per class
misclassified_data_per_class = {0: [], 1: [], 2: [], 3: []}
for instance, label in zip(misclassified_df['X'], misclassified_df['y']):
    if label in misclassified_data_per_class:
        misclassified_data_per_class[label].append(instance)

# Generate augmented data
X_augmented = []
y_augmented = []
temperature = 0.4

for label, misclassified_instances in misclassified_data_per_class.items():
    num_misclassified = len(misclassified_instances)
    instances_to_generate = samples_needed[label]
    samples_per_instance = max(1, instances_to_generate // num_misclassified)

    for instance in misclassified_instances:
        print(f"Generating data for label {label} with original instance: {instance}")
        synthetic_samples = generate_augmented_data(instance, label, temperature, samples_per_instance)
        for sample in synthetic_samples:
            print(f"Generated synthetic sample: {sample} for label {label}")
            X_augmented.append(sample)
            y_augmented.append(label)




In [ ]:
import joblib

# Number of samples needed for each class
samples_needed = {
    0: 3866 - 3120,  # Class 0: 3120 is the current count
    1: 3866 - 1007,  # Class 1: 1007 is the current count
    2: 3866 - 1116,  # Class 2: 1116 is the current count
    3: 3866 - 941    # Class 3: 941 is the current count
}

# Collect misclassified instances per class
misclassified_data_per_class = {0: [], 1: [], 2: [], 3: []}
for instance, label in zip(misclassified_df['X'], misclassified_df['y']):
    if label in misclassified_data_per_class:
        misclassified_data_per_class[label].append(instance)

# Generate augmented data
X_augmented = []
y_augmented = []
temperature = 0.4

for label, misclassified_instances in misclassified_data_per_class.items():
    num_misclassified = len(misclassified_instances)
    instances_to_generate = samples_needed[label]
    samples_per_instance = max(1, instances_to_generate // num_misclassified)

    for instance in misclassified_instances:
        print(f"Generating data for label {label} with original instance: {instance}")
        synthetic_samples = generate_augmented_data(instance, label, temperature, samples_per_instance)
        for sample in synthetic_samples:
            print(f"Generated synthetic sample: {sample} for label {label}")
            X_augmented.append(sample)
            y_augmented.append(label)

joblib.dump(X_augmented, '/content/drive/My Drive/Master_Thesis/X_augmented.pkl')
joblib.dump(y_augmented, '/content/drive/My Drive/Master_Thesis/y_augmented.pkl')

Generating data for label 0 with original instance: i am still struggling to grasp the purpose of linkedin. just returned from a shoot, feeling extremely tired and in need of a hug.


KeyboardInterrupt: 

In [ ]:
X_augmented

[]

In [ ]:
X_augmented

["i'm feeling a bit lost with linkedin lately. just got back from a long day of work, feeling completely drained and craving some comfort.",
 "i just can't seem to understand the point of twitter. just got back from a long day at work, feeling completely drained and craving some comfort.",
 "i'm feeling a bit lost when it comes to understanding the point of linkedin. just got back from a long day at work, feeling utterly exhausted and craving some comfort.",
 "i just can't seem to understand the point of twitter. just got back from a long day at work, feeling completely drained and craving some comfort.",
 "i just can't seem to understand the point of twitter. just got back from a long day at work, feeling utterly drained and craving some comfort.",
 "i'm feeling a bit lost with linkedin lately. just got back from a long day at work, completely drained and craving some comfort.",
 "i just can't seem to figure out the point of twitter. just got back from a long day at work, feeling comp

In [ ]:
y_augmented

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
# Convert augmented data to a DataFrame
X_augmented_df = pd.DataFrame(X_augmented, columns=X_train.columns)

joblib.dump(X_augmented_df, '/content/drive/My Drive/Master_Thesis/X_augmented_df.pkl')
joblib.dump(y_augmented, '/content/drive/My Drive/Master_Thesis/y_augmented.pkl')

NameError: name 'X_augmented' is not defined

In [ ]:
X_augmented_df.to_csv('/content/drive/My Drive/Master_Thesis/X_augmented_df.csv')
y_augmented.to_csv('/content/drive/My Drive/Master_Thesis/y_augmented.csv')

In [ ]:
# Combine original and augmented data
X_combined = pd.concat([X_train, X_augmented_df], ignore_index=True)
y_combined = np.concatenate([y_train, y_augmented])

# Shuffle the combined dataset
X_combined, y_combined = shuffle(X_combined, y_combined, random_state=42)
print("Augmented training set size:", X_combined.shape)

# Save the augmented data
joblib.dump(X_combined, '/content/drive/My Drive/Master_Thesis/X_combined.pkl')
joblib.dump(y_combined, '/content/drive/My Drive/Master_Thesis/y_combined.pkl')

In [ ]:
#Retrain and Evaluate the Model:

# Vectorize the combined data
X_combined_tfidf = best_tfidf_vectorizer.fit_transform(X_combined)

# Retrain the XGBoost classifier
best_xgb_classifier.fit(X_combined_tfidf, y_combined)

# Evaluate on the original test set
X_test_tfidf = best_tfidf_vectorizer.transform(X_test)
y_pred_new = best_xgb_classifier.predict(X_test_tfidf)

# Print new classification report
print(classification_report(y_test, y_pred_new))

# Update confusion matrix and other metrics as necessary

In [ ]:
###########current
import pandas as pd
import numpy as np
import os
import openai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re

# Set OpenAI API key
if 'OPENAI_API_KEY' in os.environ:
    openai.api_key = os.environ['OPENAI_API_KEY']
else:
    # If not set, fallback to the manually provided key
    openai.api_key = OPENAI_API_KEY

# Function to remove unwanted prefixes
def remove_prefix(text):
    # Define patterns to match "example 1:" and "1:" at the beginning of the string
    patterns = [r'^example \d+:', r'^\d+:', r'^\d', r'^\.']
    # Use regular expression to remove the prefixes
    for pattern in patterns:
        text = re.sub(pattern, '', text.strip()).strip()
    return text

# Generate augmented data for misclassified instances
def generate_augmented_data(instance, label, temperature, num_samples):
    augmented_data = []
    for _ in range(num_samples):
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0.4,
            max_tokens=350,
            frequency_penalty=0.6,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in generating synthetic text based on specified emotional tones."
                },
                {
                    "role": "user",
                    "content": f"Please generate reddit post without title directely give me the post that is similar to a class {label}, using the same feeling words as the following example:\n\n{instance}\n\n"
                }
            ])
        synthetic_content = response.choices[0].message.content.strip().lower()
        # Split the synthetic content into multiple sentences if they are concatenated
        synthetic_sentences = re.split(r'\d+\.\s+', synthetic_content)
        for sentence in synthetic_sentences:
            clean_sentence = remove_prefix(sentence.strip())
            if clean_sentence:  # Ensure no empty strings are added
                augmented_data.append(clean_sentence)
    return augmented_data

def save_augmented_data_to_csv(data, filepath='augmented_data_penality_06_3.csv'):
    df = pd.DataFrame(data, columns=['Temperature', 'Original', 'Label', 'Synthetic'])
    df.to_csv(filepath, index=False)

# Assuming X_sampled_df and y_sampled are defined as per previous context

# Parameters
temperatures = 0.4
# Generate augmented data for sampled misclassified instances at different temperatures
# Generate augmented data for sampled misclassified instances at different temperatures
# Generate augmented data for sampled misclassified instances at different temperatures
augmented_data_list = []
for temperature in temperatures:
    for idx, original_instance in X_sampled_df.iterrows():
        label = y_sampled[idx]
        synthetic_samples_count = 0  # Keep track of synthetic samples generated for each sentence
        synthetic_samples = generate_augmented_data(original_instance['X'], label, temperature, num_samples=2)
        print(f"Temperature: {temperature}, Original: {original_instance['X']}, Label: {label}")
        for synthetic_sample in synthetic_samples:
            print(f"Synthetic: {synthetic_sample}")
            augmented_data_list.append([temperature, original_instance['X'], label, synthetic_sample])
            synthetic_samples_count += 1
            if synthetic_samples_count >= 2:  # Stop generating samples once desired count is reached
                break
# Save the augmented data to a CSV file
save_augmented_data_to_csv(augmented_data_list)
# Function to calculate cosine similarity
def calculate_similarity(original, synthetic):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([original] + synthetic)
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    return cosine_similarities
# Read the augmented data from the CSV file
augmented_data = pd.read_csv('augmented_data_penality_06_3.csv')

# Get unique temperatures
temperatures = augmented_data['Temperature'].unique()

# Store the results
similarity_results = {temp: [] for temp in temperatures}

# Calculate cosine similarity for each original-synthetic pair
for temperature in temperatures:
    temp_data = augmented_data[augmented_data['Temperature'] == temperature]
    for original in temp_data['Original'].unique():
        synthetic_samples = temp_data[temp_data['Original'] == original]['Synthetic'].tolist()
        similarities = calculate_similarity(original, synthetic_samples)
        similarity_results[temperature].extend(similarities)

# Calculate average similarity for each temperature
average_similarities = {temp: np.mean(similarities) for temp, similarities in similarity_results.items()}

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(temperatures, list(average_similarities.values()), marker='o')
plt.title('Cosine Similarity vs. Temperature using Frequency Penality=0.6')
plt.xlabel('Temperature')
plt.ylabel('Average Cosine Similarity')
plt.grid(True)

# Add the maximum similarity value to the plot
max_similarity_temp = max(average_similarities, key=average_similarities.get)
max_similarity = average_similarities[max_similarity_temp]
plt.annotate(f"Max Similarity: {max_similarity:.2f}",
             xy=(max_similarity_temp, max_similarity),
             xytext=(max_similarity_temp, max_similarity - 0.02),  # Adjust the y-coordinate here
             arrowprops=dict(facecolor='black', shrink=0.05),
             horizontalalignment='center',
             textcoords='offset points',  # Set the coordinate system for text
             fontsize=10)  # Adjust fontsize as needed


# Save and show the plot
plt.savefig('Cosine_Similarity_vs_Temperatureaugmented_data_penality_06_3.png')
plt.show()

# Print the maximum similarity value
print(f"Max Similarity: {max_similarity:.2f} at Temperature: {max_similarity_temp}")
